# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1aac198c40>
├── 'a' --> tensor([[-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131]])
└── 'x' --> <FastTreeValue 0x7f1aac198b80>
    └── 'c' --> tensor([[ 0.0763, -0.0909,  0.9921, -0.7996],
                        [-0.4322,  2.0187,  1.3912, -1.7986],
                        [ 0.6306,  0.0748,  0.0459, -0.1235]])

In [4]:
t.a

tensor([[-0.4406, -1.0021, -0.1355],
        [ 0.8399, -1.0085, -0.1131]])

In [5]:
%timeit t.a

63.9 ns ± 0.0568 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1aac198c40>
├── 'a' --> tensor([[ 0.4904,  0.2267,  1.0148],
│                   [ 0.4124, -0.3059, -1.5594]])
└── 'x' --> <FastTreeValue 0x7f1aac198b80>
    └── 'c' --> tensor([[ 0.0763, -0.0909,  0.9921, -0.7996],
                        [-0.4322,  2.0187,  1.3912, -1.7986],
                        [ 0.6306,  0.0748,  0.0459, -0.1235]])

In [7]:
%timeit t.a = new_value

66.6 ns ± 0.148 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4406, -1.0021, -0.1355],
               [ 0.8399, -1.0085, -0.1131]]),
    x: Batch(
           c: tensor([[ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235]]),
       ),
)

In [10]:
b.a

tensor([[-0.4406, -1.0021, -0.1355],
        [ 0.8399, -1.0085, -0.1131]])

In [11]:
%timeit b.a

58 ns ± 0.0687 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7768, -0.9710, -0.5246],
               [ 0.8231, -0.3547,  0.1060]]),
    x: Batch(
           c: tensor([[ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235]]),
       ),
)

In [13]:
%timeit b.a = new_value

510 ns ± 0.159 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

807 ns ± 0.195 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 56.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f19f7428a60>
├── 'a' --> tensor([[[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]],
│           
│                   [[-0.4406, -1.0021, -0.1355],
│                    [ 0.8399, -1.0085, -0.1131]]])
└── 'x' --> <FastTreeValue 0x7f19f7428b20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.1 µs ± 80.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1a091d62b0>
├── 'a' --> tensor([[-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131],
│                   [-0.4406, -1.0021, -0.1355],
│                   [ 0.8399, -1.0085, -0.1131]])
└── 'x' --> <FastTreeValue 0x7f19f7521460>
    └── 'c' --> tensor([[ 0.0763, -0.0909,  0.9921, -0.7996],
                        [-0.4322,  2.0187,  1.3912, -1.7986],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 84.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 59.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0763, -0.0909,  0.9921, -0.7996],
                       [-0.4322,  2.0187,  1.3912, -1.7986],
                       [ 0.6306,  0.0748,  0.0459, -0.1235]],
              
                      [[ 0.0763, -0.0909,  0.9921, -0.7996],
                       [-0.4322,  2.0187,  1.3912, -1.7986],
                       [ 0.6306,  0.0748,  0.0459, -0.1235]],
              
                      [[ 0.0763, -0.0909,  0.9921, -0.7996],
                       [-0.4322,  2.0187,  1.3912, -1.7986],
                       [ 0.6306,  0.0748,  0.0459, -0.1235]],
              
                      [[ 0.0763, -0.0909,  0.9921, -0.7996],
                       [-0.4322,  2.0187,  1.3912, -1.7986],
                       [ 0.6306,  0.0748,  0.0459, -0.1235]],
              
                      [[ 0.0763, -0.0909,  0.9921, -0.7996],
                       [-0.4322,  2.0187,  1.3912, -1.7986],
                       [ 0.6306,  0.0748,  0.0459, -0.1235]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235],
                      [ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235],
                      [ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235],
                      [ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235],
                      [ 0.0763, -0.0909,  0.9921, -0.7996],
                      [-0.4322,  2.0187,  1.3912, -1.7986],
                      [ 0.6306,  0.0748,  0.0459, -0.1235],
                      [ 0.0763, -0.0909,  0.9921, -0.7996],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

321 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
